In [ ]:
import json
import pandas as pd
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy


# 1. JSON 파일에서 데이터 로드 (user_id, business_id, stars)
def load_data(filename):
    data = []
    with open(filename, "r", encoding="utf-8") as f:
        for line in f:
            review = json.loads(line.strip())
            data.append((review["user_id"], review["business_id"], review["stars"]))
    return data


# 2. 훈련 데이터 로드 및 변환
train_data = load_data("train_80.json")
train_df = pd.DataFrame(train_data, columns=["user_id", "business_id", "stars"])

# 3. 평가 데이터 로드 및 변환
eval_data = load_data("eval_20.json")
eval_df = pd.DataFrame(eval_data, columns=["user_id", "business_id", "stars"])

# 4. Surprise 형식으로 변환
reader = Reader(rating_scale=(1, 5))
train_dataset = Dataset.load_from_df(
    train_df[["user_id", "business_id", "stars"]], reader
)
eval_dataset = Dataset.load_from_df(
    eval_df[["user_id", "business_id", "stars"]], reader
)

# 5. SVD 모델 학습
svd = SVD(n_factors=20)
trainset = train_dataset.build_full_trainset()
svd.fit(trainset)

# 6. 평가 데이터셋 준비 및 예측
eval_testset = (
    eval_dataset.build_full_trainset().build_testset()
)  # (uid, iid, true_r) 형식의 리스트
predictions = svd.test(eval_testset)

# 7. 평가지표 계산
mae = accuracy.mae(predictions, verbose=True)
rmse = accuracy.rmse(predictions, verbose=True)

# 8. 결과 출력
print(f"\n[평가 결과]")
print(f"MAE  (평균 절대 오차): {mae:.4f}")
print(f"RMSE (평균 제곱근 오차): {rmse:.4f}")

FileNotFoundError: [Errno 2] No such file or directory: 'train_data.json'